# README

改进Alpaca源码:

- 原始Alpaca是自己读取并预处理数据，这里我改为dataset读取并用map函数批量预处理。
- 原始Alpaca训练7B模型需要四张A100卡，不算数据单独模型训练也需要100GB以上的显存，这里我改为bf16训练，可用一张A800(80G)训练。更改的地方包括 
    1. 模型加载，加载时设置torch_dtype
    2. 训练参数，增加bf16=True

In [1]:
import copy
import logging
import json
import io
import os
from pprint import pprint
from dataclasses import dataclass, field
from typing import Dict, Optional, Sequence
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

os.environ['CUDA_VISIBLE_DEVICES'] = '7'

In [2]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "<s>"
DEFAULT_UNK_TOKEN = "<unk>"

In [3]:
model_name_or_path = '../DataCollection/officials/Llama-2-7b'
# model_name_or_path = '../DataCollection/officials/Qwen2.5-1.5b-Instruct'

model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    torch_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    padding_side="right",
    use_fast=False,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [5]:
dataset = load_dataset('json', 
                       data_dir='/data02/hyzhang10/pengxia2/tws/data', 
                       data_files={
                           'train': 'alpaca_data_100.json', 
                        #    'test': 'alpaca_data_100.json'
                           }
                       )
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 100
    })
})


In [6]:
print(len(dataset['train']))
pprint(dataset['train'][0])

100
{'input': '',
 'instruction': 'Give three tips for staying healthy.',
 'output': '1.Eat a balanced diet and make sure to include plenty of fruits '
           'and vegetables. \n'
           '2. Exercise regularly to keep your body active and strong. \n'
           '3. Get enough sleep and maintain a consistent sleep schedule.'}


In [7]:
IGNORE_INDEX = -100
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}
prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]

def preprocess_func(example):
    source = prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
    target = f"{example['output']}{tokenizer.eos_token}"
    full_example = source + target
    full_example_tokenzied = tokenizer(full_example, return_tensors="pt",padding="longest", max_length=tokenizer.model_max_length, truncation=True)
    input_ids = full_example_tokenzied['input_ids'][0]
    labels = copy.deepcopy(input_ids)
    source_tokenzied = tokenizer(source, return_tensors="pt",padding="longest", max_length=tokenizer.model_max_length, truncation=True)
    labels[:len(source_tokenzied['input_ids'][0])] = IGNORE_INDEX
    return dict(
        input_ids=input_ids, 
        labels=labels
    )


# preprocess_func(dataset['train'][0])
train_ds = dataset['train'].map(preprocess_func, remove_columns=list(dataset['train'].features.keys()))
print(train_ds)

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 100
})


In [8]:
print([len(input_ids) for input_ids in train_ds['input_ids']])

[98, 55, 117, 136, 208, 71, 118, 302, 63, 87, 102, 50, 127, 157, 95, 158, 144, 143, 140, 62, 56, 370, 107, 405, 149, 226, 163, 256, 134, 148, 221, 78, 117, 219, 180, 86, 143, 88, 165, 110, 165, 81, 212, 134, 117, 197, 158, 137, 204, 140, 99, 217, 85, 89, 68, 74, 58, 93, 54, 60, 62, 85, 63, 89, 73, 161, 72, 155, 82, 206, 71, 82, 118, 93, 81, 82, 69, 325, 89, 133, 144, 88, 211, 329, 216, 142, 327, 220, 183, 77, 318, 89, 160, 104, 131, 69, 116, 67, 49, 76]


In [9]:
pprint(train_ds[0], width=1000)

{'input_ids': [1, 13866, 338, 385, 15278, 393, 16612, 263, 3414, 29889, 14350, 263, 2933, 393, 7128, 2486, 1614, 2167, 278, 2009, 29889, 13, 13, 2277, 29937, 2799, 4080, 29901, 13, 29954, 573, 2211, 25562, 363, 7952, 292, 9045, 29891, 29889, 13, 13, 2277, 29937, 13291, 29901, 29896, 29889, 29923, 271, 263, 6411, 8362, 652, 300, 322, 1207, 1854, 304, 3160, 20947, 310, 285, 21211, 322, 18655, 1849, 29889, 29871, 13, 29906, 29889, 1222, 6269, 895, 25704, 304, 3013, 596, 3573, 6136, 322, 4549, 29889, 29871, 13, 29941, 29889, 3617, 3307, 8709, 322, 7344, 263, 13747, 8709, 20410, 29889, 2],
 'labels': [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 29896, 29889, 29923, 271, 263, 6411, 8362, 652, 300, 322, 1207, 1854, 304, 3160, 20947, 310, 285, 21211, 322, 18655, 1849, 298

In [11]:
@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: AutoTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([torch.tensor(instance[key]) for instance in instances] for key in ("input_ids", "labels"))
        # input_ids 用 pad_token_id 补齐
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        # labels 用 IGNORE_INDEX 补齐
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id), # attention_mask 直接基于 input_ids
        )

dc = DataCollatorForSupervisedDataset(tokenizer)
# DataCollator的输入是list[dict[str, tensor]]
ret = [train_ds[index] for index in range(2,4)]
ret = dc(ret)
print([len(input_ids) for input_ids in ret['input_ids']])

[136, 136]


In [12]:
train_dataset = train_ds
eval_dataset=None
data_collator = DataCollatorForSupervisedDataset(tokenizer)

In [13]:
training_args = TrainingArguments(output_dir='./output',
                                  num_train_epochs=3,
                                  per_device_train_batch_size=2,
                                  per_device_eval_batch_size=8,
                                  gradient_accumulation_steps=8,
                                  evaluation_strategy='no',
                                  save_strategy='steps',
                                  save_steps=2000,
                                  save_total_limit=1,
                                  learning_rate=2e-5,
                                  weight_decay=0.,
                                  warmup_ratio=0.03,
                                  lr_scheduler_type='cosine',
                                  logging_steps=1,
                                  report_to=[],
                                  bf16=True
                                  )
trainer = Trainer(model=model, 
                  tokenizer=tokenizer, 
                  args=training_args, 
                  train_dataset=train_dataset,
                  eval_dataset=eval_dataset,
                  data_collator=data_collator,
                  )
train_output = trainer.train()

/data02/hyzhang10/miniconda3/envs/xp-nlp/lib/python3.12/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_41749/2745160975.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model,
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


[2024-12-16 15:00:43,755] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/data02/hyzhang10/miniconda3/envs/xp-nlp/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda-12.2/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/data02/hyzhang10/miniconda3/envs/xp-nlp/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda-12.2/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/data02/hyzhang10/miniconda3/envs/xp-nlp/compiler_compat/ld: /usr/local/cuda-12.2/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/data02/hyzhang10/miniconda3/envs/xp-nlp/compiler_compat/ld: /usr/local/cuda-12.2/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/data02/hyzhang10/miniconda3/envs/xp-nlp/compiler_compat/ld: /usr/local/cuda-12.2/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/data02/hyzhang10/miniconda3/envs/xp-nlp/compiler_compat/ld: /usr/local/cuda-12.2/lib64/libcufile.so: undefined reference to `std::chrono::_V2::ste

Step,Training Loss
1,1.463100
2,1.510700
3,1.220400
4,1.248900
5,1.209200
6,1.126600
7,1.891500
8,0.871800
9,0.741400
10,0.794900


TrainOutput(global_step=18, training_loss=0.9919749034775628, metrics={'train_runtime': 110.6316, 'train_samples_per_second': 2.712, 'train_steps_per_second': 0.163, 'total_flos': 2013363317293056.0, 'train_loss': 0.9919749034775628, 'epoch': 2.88})

In [ ]:
log_history = trainer.state.log_history
losses = [entry["loss"] for entry in log_history if "loss" in entry]

In [ ]:
plt.plot(losses)